# Emotion-to-Sound Experiment 

In [1]:
%pylab
import ipywidgets as widgets
from functools import partial
from IPython.display import display
import pandas, imp, OSC, threading
import pickle, time,socket

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
# first: select sound model 
#soundmodel = 'abstract'
soundmodel = 'vocal'
portMax = 9000
        
# Open SC regardless what model to use. 
sonmod = imp.load_source('sonmod', 'EmoSonics-soundmodels.py')
print "wait for SC to start..."
time.sleep(4); # wait until SC is started...
print "resume: establish OSC interfaces..."
"""
To run vocal model: 1. make sure the path in EmoSonics-soundmodels.py has the correct path point at the sclang. 
                    2. make sure vowel is installed in Quark
To run abstract mode: Make sure the Max program AbstractModel is in the same folder
"""
"""
Setup the OSC model here. 
"""

# SuperCollider Client
clientSC  = OSC.OSCClient(); clientSC.connect(("127.0.0.1", 57110)) # SC uses 57110 by default. 
def sc_msg(onset, msgAdr="/s_new", msgargs=["s1", 2000, 1, 0, "freq", 300, "amp", 0.5]):
    global clientSC
    bundle = OSC.OSCBundle()
    msg = OSC.OSCMessage()
    print msg
    msg.setAddress(msgAdr)
    msg.extend(msgargs)
    bundle.append(msg)
    bundle.setTimeTag(onset)
    clientSC.send(bundle)

# Max Client. 
clientMAX = OSC.OSCClient(); clientMAX.connect(("127.0.0.1", portMax)) 
def max_msg(onset, msgAdr="/s_new", msgargs=["freq", 300, "amp", 0.5]):
    global clientMAX
    bundle = OSC.OSCBundle()
    msg = OSC.OSCMessage()
    msg.setAddress(msgAdr)
    msg.extend(msgargs)
    bundle.append(msg)
    bundle.setTimeTag(onset)
    clientMAX.send(bundle)
    
if (soundmodel == "vocal") :    
    sc_msg(0, "/s_new", ["reverb", 1001, 1, 0, "outbus", 0, "room", 0.4, "mix", 0.1, "damp", 0.8]);
else: pass  
# Test sound. 
if (soundmodel == "vocal") :
    # use test tone to check if the reverb is one
    sc_msg(0, "/s_new", ["default", 1002, 1, 1]); now = time.time(); 
    sc_msg(now+0.5, "/n_free", [1002])
else: pass

"""
Three types of mapping schemes:
    1. lin -> linear 2. exp -> exponential 3. mf -> mid flatten 
"""
# TH: for vocal synth
parspec_vocal = array([ # name, min, max, scaling (lin/exp), default
#("evrate", 0.2, 4, "exp", 0.5, "Hz"),
#("irregularity", 0, 1, "lin", 0, "%"),
("dur", 0.05, 2.0, "exp", 0.4, "secs"), 
("att", 0.001, 0.5, "exp", 0.001, "secs"),
("decslope", -50, 10, "lin", -12, "dB/rm time"),
("amint",  0, 1, "lin", 0, "intensity"),
("amfreq", 1, 50, "exp", 1, "Hz"),
("pitch", 20, 85, "lin", 50, "midinote"),
("chirp", -36, 36, "mf", 0, "semitones/dur"),
("lfnfrq", 5, 50, "exp", 5, "Hz"),
("lfnint", 0, 0.5, "lin", 0, "rel. pitch"),
("vowel", 0, 4, "lin", 2.5, "uoaei"),
("voweldiff", -2.5, 2.5, "lin", 0, "delta"),
("bright", 0.2, 1, "lin", 0.5, "arb.u.")], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

# JJ: New parspec
parspec_abstract = array([ # name, min, max, scaling (lin/exp), default
#("evrate", 0.2, 4, "exp", 0.5, "Hz"),
#("irregularity", 0, 1, "lin", 0, "%"),
("dur", 0., 1., "lin", 0.5, "secs"),
("att", 0., 1., "lin", 0.3, "%"),
("desvol", 0., 1., "lin", 0.5, "dB/dur"),
("pitch", 0, 1., "lin", 0.5, "Hz"),
("chirp", 0., 1., "lin", 0.5, "semitones/dur"),
("lfndepth", 0., 1., "lin", 0., "rate"),
("lfnfreq", 0, 1., "lin", 0., "Hz"),
("amdepth", 0., 1., "lin", 0., "rate"),
("amfreq", 0., 1., "lin", 0., "Hz"),
("richness", 0., 1., "lin", 0.5, "%"),
("lpfreq", 0., 1., "lin", 0.5, "Hz")  # I wonder if this is important.       
    ], dtype=[
      ('name', 'S20'), ('min', '>f4'), ('max', '>f4'), ('scaling', 'S10'), ('default', '>f4'), ('unit', 'S20')])

"""
--------------------------------------------------------------------------------------------------------------
"""

"""
All the GUI functions:
"""
def parmap(par=("pitch", 20, 85, "lin", 50, "midinote"), val=0.5):
    mi, ma = par[1], par[2]
    if(par[3]=="lin"): return mi+(ma-mi)*val
    if(par[3]=="exp"): return mi*exp(log(ma/mi)*val)
    if(par[3]=="mf"): 
        temp = ma*pow(2*val - 1,3)
        print temp
        return temp
    

def parunmap(par=("pitch", 20, 85, "lin", 50, "midinote"), val=40):
    mi, ma = par[1], par[2]
    if(par[3]=="lin"): return (val-mi)/(ma-mi)
    if(par[3]=="exp"): return log(val/mi)/log(ma/mi)
    if(par[3]=="mf"): return (val/ma)**(1./3)/2 + 0.5
    
def parvecmap(parspec, vec):
    return array([parmap(parspec[k], v) for k,v in enumerate(vec)])

def parvecunmap(parspec, vec):
    return array([parunmap(parspec[k], v) for k,v in enumerate(vec)])    

# test code:
# print parvecunmap(parspec_vocal, parspec['default']) # get default parameters

def playevent(soundmodel, v):
    # v is unmapped vector, i.e. vector elements in [0,1]
    if(soundmodel=='vocal'):
        ps = parspec_vocal
        vec = parvecmap(ps, v);
        sc_msg(0, "/s_new", ["jj1", 1002+random.randint(900), 1,1] + 
           [x for pair in zip(ps['name'].tolist(), vec) for x in pair] );
    if(soundmodel=='abstract'):
        ps = parspec_abstract
        vec = parvecmap(ps, v);
        max_msg(0, "/s_new" , [x for pair in zip(ps['name'].tolist(), vec) for x in pair] )
        
    if(soundmodel=='physiological'):
        ps = parspec_physiological
        vec = parvecmap(ps, v);
        max_msg(0, "/s_new" , [x for pair in zip(ps['name'].tolist(), vec) for x in pair] )
        

def mutate(parent, sigma=0.1):
    d=size(parent)
    child = clip((parent + sigma*random.randn(d)), 0, 1)
    return child

def create_next_generation(parentvec, sigma, nr_of_children=4):
    return [parentvec] + [mutate(parentvec, sigma) for k in range(nr_of_children)]

def append_data(dataset, time, target, generation_counter, logsigma, parvec, submit=0, userid=-1, sound="vocal", run=-1):
    dataset.append([userid, sound, run, time, target, generation_counter, logsigma, parvec, submit])    

def savedata(dataset, prefix="user-soundmodel-run-"):
    df = pandas.DataFrame(data, columns=['uid', 'snd', 'run', 'time','target','generation','logsigma','parvec','submit'])
    df.to_csv(prefix + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)

wait for SC to start...
resume: establish OSC interfaces...
 []
 []
 []


## Experiment GUI:

In [3]:
username = "fake"
userid = 1010
soundmodel = "abstract"
log_sigma_initval = -1.0
log_sigma_step = -0.2
data = []  # time, target state, generation_counter, sigma, parvec
generation_counter = 0
v = []
target_set_time = time.time()
targetlist = ['happy', 'surprised', 'angry', 'afraid', 'disgusted', 
              'sad', 'calm', 'neutral'];
currentEmotion = 0 # use it to loop through targetlist
degreelist = ['0', '1', '2', '3','4']
class Rating_receiver:
    def __init__(self, ip = '192.168.0.3', port = 8022, num_emotion = 9, num_degree = 4):
        self.receive_address = ip, port
        self.num_emotion = num_emotion
        self.num_degree = num_degree
        self.choice = 0
        self.degree = 0
        self.logValue = -1.0
        self.count = 0
        self.run = 0
        
    def reset_settings(self):
        global generation_counter, v,  soundmodel
        print "reset!"
        generation_counter = 0 
        if(soundmodel=='vocal'):    ps = parspec_vocal
        if(soundmodel=='abstract'): ps = parspec_abstract
        if (soundmodel == 'physiological'): ps = parspec_physiological
            
        v = create_next_generation(parvecunmap(ps, ps['default']), 0.25)
        
    def spawn(self):
        print"Server Created."
        self.receiveServer = OSC.OSCServer(self.receive_address)
        print self.receive_address# create a serve to receive OSC from the tablet
        self.receiveServer.addDefaultHandlers()
        print self.receiveServer
        print self.receiveServer.address()
        
    def initilisation_handler(self, addr, tags, stuff, source):
        global userid, username,soundmodel, v, data
        print "Initialization"
        data=[]
        v =[]
        target_set_time = time.time()
        self.run = int(stuff[3])
        userid,username = stuff[1].split(',')
        soundmodel = stuff[2] 
        self.reset_settings()
        print userid,username,self.run,soundmodel
        
    def emotion_handler(self, addr, tags, stuff, source):
        self.choice = targetlist[stuff[0]]

    def degree_handler(self, addr, tags, stuff, source):
        self.degree = degreelist[stuff[0]]
        

    def log_handler(self, addr, tags, stuff, source):
        self.logValue = float(stuff[-1]/100.0)-5

        
        # This is nextVariation from Android
    def next_variation_handler(self, addr, tags, stuff, source):
        global  v,data,soundmodel,generation_counter
        generation_counter +=1
        self.degree= int(stuff[1])
        print "get next mutations"
        data.append([userid, soundmodel, self.run, time.time()-target_set_time,targetlist[self.choice], generation_counter, self.logValue, v[int(self.degree)].tolist(), 0])    
#         print "variation value ",self.degree
        #append_data(data, time.time() - target_set_time, targetlist[self.choice], generation_counter, 
        #        self.logValue, v[int(self.degree)].tolist(),  userid=userid, sound=soundmodel, run=run)
        v = create_next_generation(v[int(self.degree)], exp(self.logValue))
        self.logValue= float(stuff[0]/100.0)-5        
        
        self.degree = 0
        
    def accept_handler(self, addr, tags, stuff, source):        
        global  v,data,soundmodel,generation_counter
        print "Next Emotion"
        generation_counter +=1
        data.append([userid, soundmodel, self.run, time.time()-target_set_time,targetlist[self.choice], generation_counter, self.logValue, v[int(self.degree)].tolist(), 1])    
        self.choice= int(stuff[0])
        self.reset_settings()
        self.degree = 0

    def save_handler(self, addr, tags, stuff, source):
        global  v, data, username,generation_counter
        print "completed. thanks."
        fname = "../data/%s-%s-run%d" % (username, soundmodel, self.run) # All 3 needs to be replaced. 
        print fname
        #         savedata(data, fname)
        df = pandas.DataFrame(data, columns=['uid', 'snd', 'run', 'time','target','generation','logsigma','parvec','submit'])
        df.to_csv(fname + time.strftime("-%Y%m%d-%H%M%S") + ".csv", index=False)
        print "data saved to " + fname
        print "Please inform the operator."
        self.reset_settings()
        self.degree = 0
        
    """
    This is triggered when 0~4 is pressed. 
    """
    def mutation_selection_handler(self, addr, tags, stuff, source):
        global v,generation_counter,data, soundmodel
        generation_counter +=1
        parent = int(stuff[0])
        print "parent id", parent# This is the index of the variation. 
        
        print self.degree
#         data.append([userid, soundmodel, self.run, time.time()-target_set_time,targetlist[self.choice], 
#                generation_counter, self.logValue, v[int(self.degree)].tolist(), 0])    
        # Play sound
#         playevent(soundmodel, v[parent])

        #append_data(data, time.time()-target_set_time, targetlist[self.choice], generation_counter, 
        #        self.logValue, v[parent].tolist(), userid=userid, sound=soundmodel, run=run)
        

    def add_handler(self):
        self.receiveServer.addMsgHandler("/play", self.next_variation_handler)
        self.receiveServer.addMsgHandler("/next", self.accept_handler)
        self.receiveServer.addMsgHandler("/emo", self.emotion_handler)
        self.receiveServer.addMsgHandler("/degree", self.degree_handler)
        self.receiveServer.addMsgHandler("/save", self.save_handler)
        self.receiveServer.addMsgHandler("/init", self.initilisation_handler)
        self.receiveServer.addMsgHandler("/logChange", self.log_handler)
        self.receiveServer.addMsgHandler("/variationselection", self.mutation_selection_handler)
            
    def print_registered_func(self):
        for addr in self.receiveServer.getOSCAddressSpace():
            print addr
            
    def start(self):
        # Start OSCServer
        print "\nStarting OSCServer."
        self.emorating_oscServer = threading.Thread(target = self.receiveServer.serve_forever)
        self.emorating_oscServer.start()
        print "\nOSCServer established."
        
    def stop(self):
        # Close the OSC server
        print "\nClosing OSCServer."
        self.receiveServer.close()
        print "Waiting for Server-thread to finish"
        try:
            self.emorating_oscServer.join() ##!!!
            print "Done"
        except AttributeError:
            print AttributeError

In [4]:
# ipAddr = socket.gethostbyname(socket.getfqdn())
ipAddr = "192.168.0.17"
init_receiver = Rating_receiver (ip = ipAddr, port = 8022,num_emotion = 8, num_degree = 4)
init_receiver.spawn()
init_receiver.add_handler()
# init_receiver.print_registered_func()
# init_receiver.reset_settings()
init_receiver.start()

Server Created.
('192.168.0.17', 8022)
OSCServer v0.3.5b-5294 listening on osc://192.168.0.17:8022
('192.168.0.17', 8022)

Starting OSCServer.

OSCServer established.


In [ ]:
try:
    init_receiver.stop()
    rating_receiver.stop()
    
except:
    print "OK"

In [ ]:
ipAddr = socket.gethostbyname(socket.getfqdn())